In [2]:
import os
import tensorflow as tf
import time
import csv

In [3]:
def decode(serialized_example):
    features = tf.parse_single_example(
      serialized_example,      
      features={
          'label': tf.FixedLenFeature([], tf.int64),
          'width': tf.FixedLenFeature([], tf.int64),
          'height': tf.FixedLenFeature([], tf.int64),
          'image': tf.FixedLenFeature([], tf.string)
      })
    image = tf.decode_raw(features['image'],tf.uint8)
    #image = tf.reshape(image,[64,64,1])
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, [64,64,1])
    label = tf.cast(features['label'],tf.int64)
    
    return image,label

In [4]:
batch_size=100

train_file_name = "Shuffled_PE92_Train.tfrecord"
test_file_name = "Shuffled_PE92_Test.tfrecord"

train_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+train_file_name))
test_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+test_file_name))
total_test_batch = int(test_data_length/batch_size)+1

In [5]:

test_dataset = tf.data.TFRecordDataset("./"+test_file_name)
test_dataset = test_dataset.map(decode)
#test_dataset = test_dataset.repeat()
#test_dataset= test_dataset.shuffle(buffer_size=10000)
test_dataset = test_dataset.batch(batch_size)


In [6]:
train_dataset = tf.data.TFRecordDataset("./"+train_file_name)
train_dataset = train_dataset.map(decode)
train_dataset = train_dataset.repeat()
train_dataset= train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.batch(batch_size)

In [7]:
train_iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
test_iterator = tf.data.Iterator.from_structure(test_dataset.output_types,
                                           test_dataset.output_shapes)

train_init_op = train_iterator.make_initializer(train_dataset)
test_init_op = test_iterator.make_initializer(test_dataset)

train_image, train_label = train_iterator.get_next()
test_image, test_label = test_iterator.get_next()

train_total_batch = int(train_data_length/batch_size)
test_total_batch = int(test_data_length/batch_size)

In [8]:
input_images = tf.placeholder(tf.float32,[None,64,64,1])
input_labels = tf.placeholder(tf.int64,[None])

is_training = tf.placeholder(tf.bool)
keep_prob = tf.placeholder(tf.float32)
print(input_images)
print(input_labels)
print(is_training)
print(keep_prob)

Tensor("Placeholder:0", shape=(?, 64, 64, 1), dtype=float32)
Tensor("Placeholder_1:0", shape=(?,), dtype=int64)
Tensor("Placeholder_2:0", dtype=bool)
Tensor("Placeholder_3:0", dtype=float32)


In [9]:
W1 = tf.Variable(tf.random_normal([5,5,1,32], stddev=0.01))
L1 = tf.nn.conv2d(input_images,W1, strides=[1,1,1,1], padding = 'SAME')
print(L1)
bn1 = tf.layers.batch_normalization(L1, training=is_training)
at1 = tf.nn.relu(bn1)

pool1 =  tf.nn.max_pool(at1, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool1)

Tensor("Conv2D:0", shape=(?, 64, 64, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 32, 32, 32), dtype=float32)


In [10]:
W2 = tf.Variable(tf.random_normal([5, 5, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(pool1, W2, strides=[1,1,1,1], padding = 'SAME')
bn2 = tf.layers.batch_normalization(L2, training=is_training)
at2 = tf.nn.relu(bn2)

pool2 =  tf.nn.max_pool(at2, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool2)

Tensor("MaxPool_1:0", shape=(?, 16, 16, 64), dtype=float32)


In [11]:
W3 = tf.Variable(tf.random_normal([4, 4, 64, 128], stddev=0.01))

L3 = tf.nn.conv2d(pool2,W3, strides=[1,1,1,1], padding = 'SAME')
bn3 = tf.layers.batch_normalization(L3, training=is_training)
at3 = tf.nn.relu(bn3)

pool3 =  tf.nn.max_pool(at3, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool3)

Tensor("MaxPool_2:0", shape=(?, 8, 8, 128), dtype=float32)


In [12]:
W4 = tf.Variable(tf.random_normal([4, 4, 128, 256], stddev=0.01))

L4 = tf.nn.conv2d(pool3,W4, strides=[1,1,1,1], padding = 'SAME')
bn4 = tf.layers.batch_normalization(L4, training=is_training)
at4 = tf.nn.relu(bn4)

pool4 =  tf.nn.max_pool(at4, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool4)

Tensor("MaxPool_3:0", shape=(?, 4, 4, 256), dtype=float32)


In [13]:
W5 = tf.Variable(tf.random_normal([4*4*256, 512], stddev=0.01))
fc1 = tf.reshape(pool4,[-1,4*4*256])
fc1 = tf.matmul(fc1, W5)
bn5 =tf.layers.batch_normalization(fc1, training=is_training)
fc1 = tf.nn.relu(bn5)

fc1 = tf.nn.dropout(fc1,keep_prob)
print(fc1)

Tensor("dropout/mul:0", shape=(?, 512), dtype=float32)


In [14]:
W6 = tf.Variable(tf.random_normal([512,2350],stddev=0.01))
logit = tf.matmul(fc1, W6)
print(logit)

Tensor("MatMul_1:0", shape=(?, 2350), dtype=float32)


In [15]:
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logit, labels=input_labels))
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    optimizer = tf.train.AdamOptimizer(0.0001).minimize(loss)
loss_summ=tf.summary.scalar("Loss",loss)

In [16]:
prediction = tf.argmax(logit,1)
correct_prediction = tf.equal(prediction, input_labels)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
test_accuracy = tf.reduce_sum(tf.cast(correct_prediction, tf.float32))
acc_summ=tf.summary.scalar("Accuracy",accuracy)

In [16]:
init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
saver = tf.train.Saver()

print("Train total Batch :%d" % (train_total_batch))
with tf.Session() as sess:
    predict_file = open('./save/PE92/CNN/prediction.csv','w',encoding='utf-8',newline='')
    label_file = open('./save/PE92/CNN/labels.csv','w',encoding='utf-8',newline='')

    
    merged_summary = tf.summary.merge([loss_summ, acc_summ])
    train_writer = tf.summary.FileWriter("./logs/CNN/PE92/Train")
    test_writer = tf.summary.FileWriter("./logs/CNN/PE92/Test")
    
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    sess.run(train_init_op)
    sess.run(test_init_op)#
    #Train Model
    global_step=0
    start_time = time.time()
    for epoch in range(100):
        for i in range(train_total_batch):
        #for i in range(batch_size):
            batch_images, batch_labels = sess.run([train_image, train_label])
            sess.run(optimizer,feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:0.5, is_training:True})
            train_acc, train_loss = sess.run([accuracy,loss], feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:1.0,is_training:False})
            if(i%100==0):
                print('Epoch :%d | step %d, training accuracy: %f, train loss: %f' % (epoch,i, train_acc,train_loss))
                summary = sess.run(merged_summary, feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:1.0,is_training:False})
                train_writer.add_summary(summary,global_step)
                global_step+=1
        
        end_time = time.time()
        elapsed = end_time - start_time
        print("Time : %d : %d: :%f" %(int(elapsed/3600), int((elapsed%3600)/60), (elapsed%60)))
        
        acc_sum=0.0
        sess.run(test_init_op)
        for epoch in range(total_test_batch):
            batch_images, batch_labels = sess.run([test_image, test_label])
            try:
                test_acc, test_loss=sess.run([test_accuracy,loss], feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:1.0,is_training:False})
            except tf.errors.OutOfRangeError:
                print("Out of Data ierators")
            acc_sum+=test_acc
        test_acc = acc_sum/test_data_length
        print("Test Acc : %f, Test Loss :%f" %(test_acc,test_loss))
        
        
    print("Training is done")
    save_path = saver.save(sess, "save/PE92/CNN/CNN_PE92.ckpt")
    
    acc_sum=0.0
    sess.run(test_init_op)
    for epoch in range(total_test_batch):
        batch_images, batch_labels = sess.run([test_image, test_label])
        try:
            test_acc, test_loss=sess.run([test_accuracy,loss], feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:1.0,is_training:False})
        except tf.errors.OutOfRangeError:
            print("Out of Data ierators")
        acc_sum+=test_acc
    test_acc = acc_sum/test_data_length
    print("Test Acc : %f, Test Loss :%f" %(test_acc,test_loss))

    
    #Test Model
    coord.request_stop()
    coord.join(threads)
    predict_file.close()
    label_file.close()
    train_writer.close()
    test_writer.close()

Train total Batch :2002
Epoch :0 | step 0, training accuracy: 0.000000, train loss: 7.761586
Epoch :0 | step 100, training accuracy: 0.000000, train loss: 7.762809
Epoch :0 | step 200, training accuracy: 0.000000, train loss: 7.664160
Epoch :0 | step 300, training accuracy: 0.000000, train loss: 7.415390
Epoch :0 | step 400, training accuracy: 0.010000, train loss: 6.668140
Epoch :0 | step 500, training accuracy: 0.030000, train loss: 6.209005
Epoch :0 | step 600, training accuracy: 0.090000, train loss: 5.753403
Epoch :0 | step 700, training accuracy: 0.080000, train loss: 5.592471
Epoch :0 | step 800, training accuracy: 0.100000, train loss: 5.329439
Epoch :0 | step 900, training accuracy: 0.240000, train loss: 4.923846
Epoch :0 | step 1000, training accuracy: 0.240000, train loss: 4.818788
Epoch :0 | step 1100, training accuracy: 0.280000, train loss: 4.518916
Epoch :0 | step 1200, training accuracy: 0.340000, train loss: 4.304599
Epoch :0 | step 1300, training accuracy: 0.310000, t

Epoch :5 | step 500, training accuracy: 0.970000, train loss: 0.187156
Epoch :5 | step 600, training accuracy: 0.970000, train loss: 0.198069
Epoch :5 | step 700, training accuracy: 1.000000, train loss: 0.094320
Epoch :5 | step 800, training accuracy: 0.980000, train loss: 0.107529
Epoch :5 | step 900, training accuracy: 0.970000, train loss: 0.121838
Epoch :5 | step 1000, training accuracy: 0.970000, train loss: 0.301674
Epoch :5 | step 1100, training accuracy: 0.970000, train loss: 0.128014
Epoch :5 | step 1200, training accuracy: 0.970000, train loss: 0.144907
Epoch :5 | step 1300, training accuracy: 1.000000, train loss: 0.069412
Epoch :5 | step 1400, training accuracy: 0.980000, train loss: 0.104921
Epoch :5 | step 1500, training accuracy: 0.980000, train loss: 0.122800
Epoch :5 | step 1600, training accuracy: 0.980000, train loss: 0.139577
Epoch :5 | step 1700, training accuracy: 0.980000, train loss: 0.107791
Epoch :5 | step 1800, training accuracy: 0.980000, train loss: 0.1642

Epoch :10 | step 1000, training accuracy: 0.990000, train loss: 0.027582
Epoch :10 | step 1100, training accuracy: 1.000000, train loss: 0.022674
Epoch :10 | step 1200, training accuracy: 1.000000, train loss: 0.029988
Epoch :10 | step 1300, training accuracy: 1.000000, train loss: 0.022863
Epoch :10 | step 1400, training accuracy: 1.000000, train loss: 0.023705
Epoch :10 | step 1500, training accuracy: 0.990000, train loss: 0.041569
Epoch :10 | step 1600, training accuracy: 1.000000, train loss: 0.025268
Epoch :10 | step 1700, training accuracy: 0.970000, train loss: 0.100666
Epoch :10 | step 1800, training accuracy: 0.980000, train loss: 0.054008
Epoch :10 | step 1900, training accuracy: 1.000000, train loss: 0.026095
Epoch :10 | step 2000, training accuracy: 1.000000, train loss: 0.031061
Time : 0 : 32: :35.028821
Test Acc : 0.962438, Test Loss :0.000395
Epoch :11 | step 0, training accuracy: 0.990000, train loss: 0.041178
Epoch :11 | step 100, training accuracy: 1.000000, train los

Epoch :15 | step 1400, training accuracy: 1.000000, train loss: 0.017865
Epoch :15 | step 1500, training accuracy: 1.000000, train loss: 0.024103
Epoch :15 | step 1600, training accuracy: 1.000000, train loss: 0.015795
Epoch :15 | step 1700, training accuracy: 1.000000, train loss: 0.022586
Epoch :15 | step 1800, training accuracy: 1.000000, train loss: 0.013205
Epoch :15 | step 1900, training accuracy: 1.000000, train loss: 0.011802
Epoch :15 | step 2000, training accuracy: 0.990000, train loss: 0.022741
Time : 0 : 47: :25.150733
Test Acc : 0.964959, Test Loss :0.000176
Epoch :16 | step 0, training accuracy: 1.000000, train loss: 0.013896
Epoch :16 | step 100, training accuracy: 1.000000, train loss: 0.006496
Epoch :16 | step 200, training accuracy: 0.990000, train loss: 0.026018
Epoch :16 | step 300, training accuracy: 1.000000, train loss: 0.009953
Epoch :16 | step 400, training accuracy: 1.000000, train loss: 0.007731
Epoch :16 | step 500, training accuracy: 0.980000, train loss: 0

Epoch :20 | step 1800, training accuracy: 1.000000, train loss: 0.004293
Epoch :20 | step 1900, training accuracy: 1.000000, train loss: 0.005613
Epoch :20 | step 2000, training accuracy: 1.000000, train loss: 0.004510
Time : 1 : 2: :11.864450
Test Acc : 0.966583, Test Loss :0.000086
Epoch :21 | step 0, training accuracy: 1.000000, train loss: 0.004321
Epoch :21 | step 100, training accuracy: 1.000000, train loss: 0.006079
Epoch :21 | step 200, training accuracy: 1.000000, train loss: 0.001864
Epoch :21 | step 300, training accuracy: 1.000000, train loss: 0.006084
Epoch :21 | step 400, training accuracy: 1.000000, train loss: 0.007942
Epoch :21 | step 500, training accuracy: 1.000000, train loss: 0.009499
Epoch :21 | step 600, training accuracy: 1.000000, train loss: 0.005857
Epoch :21 | step 700, training accuracy: 1.000000, train loss: 0.005444
Epoch :21 | step 800, training accuracy: 1.000000, train loss: 0.005999
Epoch :21 | step 900, training accuracy: 1.000000, train loss: 0.0079

...

Epoch :83 | step 100, training accuracy: 1.000000, train loss: 0.000152
Epoch :83 | step 200, training accuracy: 1.000000, train loss: 0.000218
Epoch :83 | step 300, training accuracy: 1.000000, train loss: 0.000519
Epoch :83 | step 400, training accuracy: 1.000000, train loss: 0.000060
Epoch :83 | step 500, training accuracy: 1.000000, train loss: 0.000144
Epoch :83 | step 600, training accuracy: 1.000000, train loss: 0.000079
Epoch :83 | step 700, training accuracy: 1.000000, train loss: 0.000191
Epoch :83 | step 800, training accuracy: 1.000000, train loss: 0.000303
Epoch :83 | step 900, training accuracy: 1.000000, train loss: 0.000064
Epoch :83 | step 1000, training accuracy: 1.000000, train loss: 0.000028
Epoch :83 | step 1100, training accuracy: 1.000000, train loss: 0.000357
Epoch :83 | step 1200, training accuracy: 1.000000, train loss: 0.000187
Epoch :83 | step 1300, training accuracy: 1.000000, train loss: 0.000062
Epoch :83 | step 1400, training accuracy: 1.000000, train lo

Epoch :88 | step 500, training accuracy: 1.000000, train loss: 0.006186
Epoch :88 | step 600, training accuracy: 1.000000, train loss: 0.000524
Epoch :88 | step 700, training accuracy: 1.000000, train loss: 0.000195
Epoch :88 | step 800, training accuracy: 1.000000, train loss: 0.000466
Epoch :88 | step 900, training accuracy: 1.000000, train loss: 0.000170
Epoch :88 | step 1000, training accuracy: 1.000000, train loss: 0.000243
Epoch :88 | step 1100, training accuracy: 1.000000, train loss: 0.000111
Epoch :88 | step 1200, training accuracy: 1.000000, train loss: 0.000216
Epoch :88 | step 1300, training accuracy: 1.000000, train loss: 0.000054
Epoch :88 | step 1400, training accuracy: 1.000000, train loss: 0.000052
Epoch :88 | step 1500, training accuracy: 1.000000, train loss: 0.000183
Epoch :88 | step 1600, training accuracy: 1.000000, train loss: 0.000049
Epoch :88 | step 1700, training accuracy: 1.000000, train loss: 0.000078
Epoch :88 | step 1800, training accuracy: 1.000000, trai

Epoch :93 | step 900, training accuracy: 1.000000, train loss: 0.000684
Epoch :93 | step 1000, training accuracy: 1.000000, train loss: 0.000028
Epoch :93 | step 1100, training accuracy: 1.000000, train loss: 0.000451
Epoch :93 | step 1200, training accuracy: 1.000000, train loss: 0.000057
Epoch :93 | step 1300, training accuracy: 1.000000, train loss: 0.000038
Epoch :93 | step 1400, training accuracy: 1.000000, train loss: 0.000013
Epoch :93 | step 1500, training accuracy: 1.000000, train loss: 0.000094
Epoch :93 | step 1600, training accuracy: 1.000000, train loss: 0.000066
Epoch :93 | step 1700, training accuracy: 1.000000, train loss: 0.000363
Epoch :93 | step 1800, training accuracy: 1.000000, train loss: 0.000354
Epoch :93 | step 1900, training accuracy: 1.000000, train loss: 0.000240
Epoch :93 | step 2000, training accuracy: 1.000000, train loss: 0.000020
Time : 4 : 38: :10.068617
Test Acc : 0.972950, Test Loss :0.000000
Epoch :94 | step 0, training accuracy: 1.000000, train los

Epoch :98 | step 1300, training accuracy: 1.000000, train loss: 0.000031
Epoch :98 | step 1400, training accuracy: 1.000000, train loss: 0.000266
Epoch :98 | step 1500, training accuracy: 1.000000, train loss: 0.000067
Epoch :98 | step 1600, training accuracy: 1.000000, train loss: 0.000074
Epoch :98 | step 1700, training accuracy: 1.000000, train loss: 0.000022
Epoch :98 | step 1800, training accuracy: 1.000000, train loss: 0.000032
Epoch :98 | step 1900, training accuracy: 1.000000, train loss: 0.000045
Epoch :98 | step 2000, training accuracy: 1.000000, train loss: 0.000019
Time : 4 : 52: :57.159356
Test Acc : 0.974403, Test Loss :0.000000
Epoch :99 | step 0, training accuracy: 1.000000, train loss: 0.000029
Epoch :99 | step 100, training accuracy: 1.000000, train loss: 0.000075
Epoch :99 | step 200, training accuracy: 1.000000, train loss: 0.000098
Epoch :99 | step 300, training accuracy: 1.000000, train loss: 0.000058
Epoch :99 | step 400, training accuracy: 1.000000, train loss: 

OutOfRangeError: End of sequence
	 [[Node: IteratorGetNext_1 = IteratorGetNext[output_shapes=[[?,64,64,1], [?]], output_types=[DT_FLOAT, DT_INT64], _device="/job:localhost/replica:0/task:0/device:CPU:0"](Iterator_1)]]

Caused by op 'IteratorGetNext_1', defined at:
  File "C:\Users\JIN\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\JIN\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\JIN\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Users\JIN\Anaconda3\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\JIN\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "C:\Users\JIN\Anaconda3\envs\tensorflow\lib\site-packages\zmq\eventloop\ioloop

In [17]:
train_file_name = "Shuffled_HanDB_Train.tfrecord"
test_file_name = "Shuffled_HanDB_Test.tfrecord"

train_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+train_file_name))
test_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+test_file_name))

train_dataset = tf.data.TFRecordDataset("./"+train_file_name)
train_dataset = train_dataset.map(decode)
train_dataset = train_dataset.repeat()
train_dataset= train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.batch(batch_size)

test_dataset = tf.data.TFRecordDataset("./"+test_file_name)
test_dataset = test_dataset.map(decode)
test_dataset = test_dataset.repeat()
test_dataset= test_dataset.shuffle(buffer_size=10000)
test_dataset = test_dataset.batch(batch_size)

total_test_batch = int(test_data_length/batch_size)+1

train_iterator = tf.data.Iterator.from_structure(train_dataset.output_types, train_dataset.output_shapes)
test_iterator = tf.data.Iterator.from_structure(test_dataset.output_types, test_dataset.output_shapes)

train_init_op = train_iterator.make_initializer(train_dataset)
test_init_op = test_iterator.make_initializer(test_dataset)

train_image, train_label = train_iterator.get_next()
test_image, test_label = test_iterator.get_next()

train_total_batch = int(train_data_length/batch_size)
test_total_batch = int(test_data_length/batch_size)

In [18]:
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logit, labels=input_labels))
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    optimizer = tf.train.AdamOptimizer(0.0001).minimize(loss)
loss_summ2=tf.summary.scalar("Loss",loss)

prediction = tf.argmax(logit,1)
correct_prediction = tf.equal(prediction, input_labels)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
test_accuracy = tf.reduce_sum(tf.cast(correct_prediction, tf.float32))
acc_summ2=tf.summary.scalar("Accuracy",accuracy)

In [20]:
init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
saver = tf.train.Saver()

print("Train total Batch :%d" % (train_total_batch))
with tf.Session() as sess:

    
    merged_summary2 = tf.summary.merge([loss_summ2, acc_summ2])
    train_writer = tf.summary.FileWriter("./logs/CNN/HanDB/Train")
    test_writer = tf.summary.FileWriter("./logs/CNN/HanDB/Test")
    
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    sess.run(train_init_op)
    sess.run(test_init_op)#
    #Train Model
    
    start_time = time.time()
    global_step=0
    for epoch in range(100):
        for i in range(train_total_batch):
        #for i in range(batch_size):
            batch_images, batch_labels = sess.run([train_image, train_label])
            sess.run(optimizer,feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:0.5, is_training:True})
            train_acc, train_loss=sess.run([accuracy,loss], feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:1.0,is_training:False})
            if(i%100==0):
                print('Epoch :%d | step %d, training accuracy: %f, train loss: %f' % (epoch,i, train_acc,train_loss))
                summary = sess.run(merged_summary2, feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:1.0,is_training:False})
                train_writer.add_summary(summary,global_step)
                global_step+=1
    
        end_time = time.time()
        elapsed = end_time - start_time
        print("Time : %d : %d: :%f" %(int(elapsed/3600), int((elapsed%3600)/60), (elapsed%60)))
        acc_sum=0.0
        sess.run(test_init_op)
        for epoch in range(total_test_batch):
            batch_images, batch_labels = sess.run([test_image, test_label])
            try:
                test_acc, test_loss=sess.run([test_accuracy,loss], feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:1.0,is_training:False})
            
            except tf.errors.OutOfRangeError:
                print("Out of Data ierators")
            acc_sum+=test_acc
        test_acc = acc_sum/test_data_length
        print("Test Acc : %f, Test Loss :%f" %(test_acc,test_loss))
        
    print("Training is done")
    save_path = saver.save(sess, "save\HanDB\CNN\CNN_HanDB.ckpt")
    acc_sum=0.0
    sess.run(test_init_op)
    for epoch in range(total_test_batch):
        batch_images, batch_labels = sess.run([test_image, test_label])
        try:
            test_acc, test_loss=sess.run([test_accuracy,loss], feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:1.0,is_training:False})
            loss_sum+=test_loss*len(batch_cho)

        except tf.errors.OutOfRangeError:
            print("Out of Data ierators")
        acc_sum+=test_acc
        loss_sum=loss_sum/test_data_length
    test_acc = acc_sum/test_data_length
    print("Test Acc : %f, Test Loss :%f" %(test_acc,test_loss))

    
    #Test Model
    coord.request_stop()
    coord.join(threads)
    train_writer.close()
    test_writer.close()

Train total Batch :6658
Epoch :0 | step 0, training accuracy: 0.000000, train loss: 7.756993
Epoch :0 | step 100, training accuracy: 0.000000, train loss: 7.353539
Epoch :0 | step 200, training accuracy: 0.010000, train loss: 6.804028
Epoch :0 | step 300, training accuracy: 0.000000, train loss: 6.813693
Epoch :0 | step 400, training accuracy: 0.120000, train loss: 5.583023
Epoch :0 | step 500, training accuracy: 0.210000, train loss: 5.936391
Epoch :0 | step 600, training accuracy: 0.280000, train loss: 5.079480
Epoch :0 | step 700, training accuracy: 0.190000, train loss: 5.145730
Epoch :0 | step 800, training accuracy: 0.320000, train loss: 4.816137
Epoch :0 | step 900, training accuracy: 0.360000, train loss: 4.945737
Epoch :0 | step 1000, training accuracy: 0.430000, train loss: 4.094304
Epoch :0 | step 1100, training accuracy: 0.350000, train loss: 4.394382
Epoch :0 | step 1200, training accuracy: 0.490000, train loss: 3.925754
Epoch :0 | step 1300, training accuracy: 0.440000, t

Epoch :1 | step 4600, training accuracy: 0.950000, train loss: 0.162274
Epoch :1 | step 4700, training accuracy: 0.940000, train loss: 0.168602
Epoch :1 | step 4800, training accuracy: 0.950000, train loss: 0.279638
Epoch :1 | step 4900, training accuracy: 0.960000, train loss: 0.230132
Epoch :1 | step 5000, training accuracy: 0.960000, train loss: 0.227741
Epoch :1 | step 5100, training accuracy: 0.980000, train loss: 0.119970
Epoch :1 | step 5200, training accuracy: 0.950000, train loss: 0.195250
Epoch :1 | step 5300, training accuracy: 0.960000, train loss: 0.143314
Epoch :1 | step 5400, training accuracy: 0.950000, train loss: 0.165001
Epoch :1 | step 5500, training accuracy: 0.960000, train loss: 0.166274
Epoch :1 | step 5600, training accuracy: 0.980000, train loss: 0.138473
Epoch :1 | step 5700, training accuracy: 0.960000, train loss: 0.184911
Epoch :1 | step 5800, training accuracy: 0.930000, train loss: 0.185530
Epoch :1 | step 5900, training accuracy: 0.950000, train loss: 0

Epoch :3 | step 2500, training accuracy: 0.990000, train loss: 0.079558
Epoch :3 | step 2600, training accuracy: 1.000000, train loss: 0.031405
Epoch :3 | step 2700, training accuracy: 0.960000, train loss: 0.113224
Epoch :3 | step 2800, training accuracy: 0.950000, train loss: 0.200542
Epoch :3 | step 2900, training accuracy: 1.000000, train loss: 0.085715
Epoch :3 | step 3000, training accuracy: 0.960000, train loss: 0.120310
Epoch :3 | step 3100, training accuracy: 0.980000, train loss: 0.075930
Epoch :3 | step 3200, training accuracy: 0.970000, train loss: 0.088848
Epoch :3 | step 3300, training accuracy: 1.000000, train loss: 0.040231
Epoch :3 | step 3400, training accuracy: 0.990000, train loss: 0.058450
Epoch :3 | step 3500, training accuracy: 0.960000, train loss: 0.105732
Epoch :3 | step 3600, training accuracy: 0.990000, train loss: 0.066700
Epoch :3 | step 3700, training accuracy: 0.930000, train loss: 0.165574
Epoch :3 | step 3800, training accuracy: 0.920000, train loss: 0

Epoch :5 | step 400, training accuracy: 0.970000, train loss: 0.190090
Epoch :5 | step 500, training accuracy: 0.990000, train loss: 0.038451
Epoch :5 | step 600, training accuracy: 1.000000, train loss: 0.036765
Epoch :5 | step 700, training accuracy: 0.980000, train loss: 0.142328
Epoch :5 | step 800, training accuracy: 0.990000, train loss: 0.042609
Epoch :5 | step 900, training accuracy: 0.970000, train loss: 0.087267
Epoch :5 | step 1000, training accuracy: 0.990000, train loss: 0.033050
Epoch :5 | step 1100, training accuracy: 0.990000, train loss: 0.070544
Epoch :5 | step 1200, training accuracy: 0.990000, train loss: 0.040573
Epoch :5 | step 1300, training accuracy: 1.000000, train loss: 0.019174
Epoch :5 | step 1400, training accuracy: 1.000000, train loss: 0.029046
Epoch :5 | step 1500, training accuracy: 1.000000, train loss: 0.028117
Epoch :5 | step 1600, training accuracy: 0.970000, train loss: 0.088139
Epoch :5 | step 1700, training accuracy: 0.960000, train loss: 0.09791

Epoch :6 | step 5100, training accuracy: 0.980000, train loss: 0.036119
Epoch :6 | step 5200, training accuracy: 0.970000, train loss: 0.119119
Epoch :6 | step 5300, training accuracy: 1.000000, train loss: 0.022126
Epoch :6 | step 5400, training accuracy: 0.990000, train loss: 0.025544
Epoch :6 | step 5500, training accuracy: 0.960000, train loss: 0.224685
Epoch :6 | step 5600, training accuracy: 1.000000, train loss: 0.029305
Epoch :6 | step 5700, training accuracy: 0.990000, train loss: 0.056889
Epoch :6 | step 5800, training accuracy: 0.960000, train loss: 0.110103
Epoch :6 | step 5900, training accuracy: 0.970000, train loss: 0.113701
Epoch :6 | step 6000, training accuracy: 1.000000, train loss: 0.020893
Epoch :6 | step 6100, training accuracy: 0.950000, train loss: 0.167018
Epoch :6 | step 6200, training accuracy: 0.970000, train loss: 0.122595
Epoch :6 | step 6300, training accuracy: 1.000000, train loss: 0.029106
Epoch :6 | step 6400, training accuracy: 1.000000, train loss: 0

...

Epoch :10 | step 900, training accuracy: 0.970000, train loss: 0.087673
Epoch :10 | step 1000, training accuracy: 1.000000, train loss: 0.011219
Epoch :10 | step 1100, training accuracy: 1.000000, train loss: 0.007171
Epoch :10 | step 1200, training accuracy: 0.990000, train loss: 0.026430
Epoch :10 | step 1300, training accuracy: 1.000000, train loss: 0.004455
Epoch :10 | step 1400, training accuracy: 0.980000, train loss: 0.025610
Epoch :10 | step 1500, training accuracy: 1.000000, train loss: 0.009131
Epoch :10 | step 1600, training accuracy: 1.000000, train loss: 0.011528
Epoch :10 | step 1700, training accuracy: 0.980000, train loss: 0.028781
Epoch :10 | step 1800, training accuracy: 1.000000, train loss: 0.006352
Epoch :10 | step 1900, training accuracy: 0.980000, train loss: 0.049772
Epoch :10 | step 2000, training accuracy: 0.980000, train loss: 0.043573
Epoch :10 | step 2100, training accuracy: 1.000000, train loss: 0.010238
Epoch :10 | step 2200, training accuracy: 0.990000, 

Epoch :11 | step 5400, training accuracy: 0.990000, train loss: 0.051017
Epoch :11 | step 5500, training accuracy: 0.980000, train loss: 0.034180
Epoch :11 | step 5600, training accuracy: 1.000000, train loss: 0.022334
Epoch :11 | step 5700, training accuracy: 1.000000, train loss: 0.006063
Epoch :11 | step 5800, training accuracy: 1.000000, train loss: 0.015148
Epoch :11 | step 5900, training accuracy: 0.990000, train loss: 0.032867
Epoch :11 | step 6000, training accuracy: 0.990000, train loss: 0.038137
Epoch :11 | step 6100, training accuracy: 0.990000, train loss: 0.029099
Epoch :11 | step 6200, training accuracy: 0.990000, train loss: 0.023481
Epoch :11 | step 6300, training accuracy: 1.000000, train loss: 0.013386
Epoch :11 | step 6400, training accuracy: 0.980000, train loss: 0.188185
Epoch :11 | step 6500, training accuracy: 1.000000, train loss: 0.009157
Epoch :11 | step 6600, training accuracy: 0.990000, train loss: 0.027981
Time : 1 : 58: :27.178507
Test Acc : 0.959195, Test

Epoch :13 | step 3100, training accuracy: 1.000000, train loss: 0.008439
Epoch :13 | step 3200, training accuracy: 1.000000, train loss: 0.003444
Epoch :13 | step 3300, training accuracy: 0.990000, train loss: 0.012123
Epoch :13 | step 3400, training accuracy: 1.000000, train loss: 0.013218
Epoch :13 | step 3500, training accuracy: 1.000000, train loss: 0.006887
Epoch :13 | step 3600, training accuracy: 0.990000, train loss: 0.016163
Epoch :13 | step 3700, training accuracy: 1.000000, train loss: 0.007453
Epoch :13 | step 3800, training accuracy: 1.000000, train loss: 0.016308
Epoch :13 | step 3900, training accuracy: 0.990000, train loss: 0.044247
Epoch :13 | step 4000, training accuracy: 0.980000, train loss: 0.027436
Epoch :13 | step 4100, training accuracy: 1.000000, train loss: 0.007058
Epoch :13 | step 4200, training accuracy: 1.000000, train loss: 0.008827
Epoch :13 | step 4300, training accuracy: 0.990000, train loss: 0.027304
Epoch :13 | step 4400, training accuracy: 0.990000,

Epoch :15 | step 800, training accuracy: 0.990000, train loss: 0.015748
Epoch :15 | step 900, training accuracy: 1.000000, train loss: 0.017045
Epoch :15 | step 1000, training accuracy: 0.990000, train loss: 0.034371
Epoch :15 | step 1100, training accuracy: 1.000000, train loss: 0.005645
Epoch :15 | step 1200, training accuracy: 0.980000, train loss: 0.073824
Epoch :15 | step 1300, training accuracy: 1.000000, train loss: 0.005769
Epoch :15 | step 1400, training accuracy: 0.980000, train loss: 0.026448
Epoch :15 | step 1500, training accuracy: 0.990000, train loss: 0.034510
Epoch :15 | step 1600, training accuracy: 0.990000, train loss: 0.021198
Epoch :15 | step 1700, training accuracy: 1.000000, train loss: 0.003531
Epoch :15 | step 1800, training accuracy: 1.000000, train loss: 0.006255
Epoch :15 | step 1900, training accuracy: 0.990000, train loss: 0.041358
Epoch :15 | step 2000, training accuracy: 0.980000, train loss: 0.034136
Epoch :15 | step 2100, training accuracy: 1.000000, t

KeyboardInterrupt: 

In [ ]:
def read_and_decode(filename_queue, batch_size):
    tf_record_reader=tf.TFRecordReader()
    _, tf_record_serialized = tf_record_reader.read(tf_record_filename_queue)

    tf_record_features = tf.parse_single_example(
        tf_record_serialized,
        features={
            'label': tf.FixedLenFeature([], tf.int64),
            'width': tf.FixedLenFeature([], tf.int64),
            'height': tf.FixedLenFeature([], tf.int64),
            'image': tf.FixedLenFeature([], tf.string),
        })
    tf_record_image = tf.decode_raw(tf_record_features['image'],tf.uint8)
    
    tf_record_image = tf.cast(tf_record_image, tf.float32)

    tf_record_image = tf.reshape(tf_record_image, [64,64,1])

    tf_record_label = tf.cast(tf_record_features['label'],tf.int64)
    
    images, labels = tf.train.shuffle_batch([tf_record_image, tf_record_label],
                                         batch_size = batch_size,
                                         capacity=20*batch_size,
                                         min_after_dequeue=5*batch_size )
    return images, labels

In [ ]:
train_file_name = "Shuffled_PE92_Train.tfrecord"
test_file_name = "Shuffled_PE92_Test.tfrecord"
batch_size=100
train_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+train_file_name))
test_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+test_file_name))

train_dataset = tf.data.TFRecordDataset("./"+train_file_name)
train_dataset = train_dataset.map(decode)
train_dataset = train_dataset.repeat()
train_dataset= train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.batch(batch_size)

test_dataset = tf.data.TFRecordDataset("./"+test_file_name)
test_dataset = test_dataset.map(decode)
test_dataset = test_dataset.repeat()
test_dataset= test_dataset.shuffle(buffer_size=10000)
test_dataset = test_dataset.batch(batch_size)

total_test_batch = int(test_data_length/batch_size)+1

train_iterator = tf.data.Iterator.from_structure(train_dataset.output_types, train_dataset.output_shapes)
test_iterator = tf.data.Iterator.from_structure(test_dataset.output_types, test_dataset.output_shapes)

train_init_op = train_iterator.make_initializer(train_dataset)
test_init_op = test_iterator.make_initializer(test_dataset)

train_image, train_label = train_iterator.get_next()
test_image, test_label = test_iterator.get_next()

train_total_batch = int(train_data_length/batch_size)
test_total_batch = int(test_data_length/batch_size)

In [ ]:
input_images = tf.placeholder(tf.float32,[None,64,64,1])
input_labels = tf.placeholder(tf.int64,[None])
keep_prob2 = tf.placeholder(tf.float32)
print(input_images)
print(input_labels)
print(keep_prob2)

W_1 = tf.Variable(tf.random_normal([5,5,1,32], stddev=0.01))
L_1 = tf.nn.conv2d(input_images, W_1, strides = [1,1,1,1], padding = 'SAME')
#at1 = tf.nn.relu(L1)
B1 = tf.Variable(tf.constant(0.1,shape=[32]))
L_1 = L_1 + B1
at_1 = tf.nn.relu(L_1)
pool_1 =  tf.nn.max_pool(at_1, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool1)

W_2 = tf.Variable(tf.random_normal([5, 5, 32, 64], stddev=0.01))
L_2 = tf.nn.conv2d(pool_1,W_2, strides=[1,1,1,1], padding = 'SAME')
#at2 = tf.nn.relu(L2)
B2 = tf.Variable(tf.constant(0.1,shape=[64]))
L_2 = L_2 + B2
at_2 = tf.nn.relu(L_2)
pool_2 =  tf.nn.max_pool(at_2, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

W_3 = tf.Variable(tf.random_normal([4, 4, 64, 128], stddev=0.01))
B3 = tf.Variable(tf.constant(0.1,shape=[128]))

L_3 = tf.nn.conv2d(pool_2,W_3, strides=[1,1,1,1], padding = 'SAME')
#t3 = tf.nn.relu(L3)
at_3 = tf.nn.relu(L_3 + B3)

pool_3 =  tf.nn.max_pool(at_3, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

W_4 = tf.Variable(tf.random_normal([4, 4, 128, 256], stddev=0.01))
B3 = tf.Variable(tf.constant(0.1,shape=[256]))

L_4 = tf.nn.conv2d(pool_3,W_4, strides=[1,1,1,1], padding = 'SAME')
#at4 = tf.nn.relu(L4)
at_4 = tf.nn.relu(L_4 + B3)

pool_4 =  tf.nn.max_pool(at_4, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

W_5 = tf.Variable(tf.random_normal([4*4*256, 512], stddev=0.01))
fc_1 = tf.reshape(pool_4,[-1,4*4*256])
fb_1 = tf.Variable(tf.constant(0.1, shape=[512]))

fc_1 = tf.matmul(fc_1, W_5)
#fc1 = tf.nn.relu(fc1)
fc_1 = tf.nn.relu(fc_1+fb_1)

fc_1 = tf.nn.dropout(fc_1,keep_prob2)
print(fc1)

W_6 = tf.Variable(tf.random_normal([512,2350],stddev=0.01))
fb_2 = tf.Variable(tf.constant(0.1,shape=[2350]))
#logit = tf.matmul(fc1, W6)
logit2 = tf.matmul(fc_1, W_6)+fb_2


loss2 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logit2, labels=input_labels))
optimizer2 = tf.train.AdamOptimizer(0.0001).minimize(loss2)
loss_summ3=tf.summary.scalar("Loss",loss2)


correct_prediction2 = tf.equal(tf.argmax(logit2,1),input_labels)
accuracy2 = tf.reduce_mean(tf.cast(correct_prediction2, tf.float32))
test_accuracy2 = tf.reduce_sum(tf.cast(correct_prediction2, tf.float32))
acc_summ3=tf.summary.scalar("Accuracy",accuracy2)

In [ ]:
init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
saver = tf.train.Saver()

print("Train total Batch :%d" % (train_total_batch))
with tf.Session() as sess:
    
    merged_summary3 = tf.summary.merge([loss_summ3, acc_summ3])
    train_writer = tf.summary.FileWriter("./logs/Basic/PE92/Train")
    
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    sess.run(train_init_op)
    sess.run(test_init_op)#
    #Train Model
    
    start_time = time.time()
    global_step=0
    for epoch in range(100):
        for i in range(train_total_batch):
        #for i in range(batch_size):
            batch_images, batch_labels = sess.run([train_image, train_label])
            sess.run(optimizer2,feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob2:0.5})
            
            train_acc, train_loss = sess.run([accuracy2, loss2],feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob2:1.0})
            
            if(i%100==0):
                print('Epoch :%d | step %d, training accuracy: %f, train loss: %f' % (epoch,i, train_acc,train_loss))
                summary = sess.run(merged_summary3, feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob2:1.0})
                train_writer.add_summary(summary,global_step)
                global_step+=1
                
        end_time = time.time()
        elapsed = end_time - start_time
        print("Time : %d : %d: :%f" %(int(elapsed/3600), int((elapsed%3600)/60), (elapsed%60)))
        
        acc_sum=0.0
        sess.run(test_init_op)
        for epoch in range(total_test_batch):
            batch_images, batch_labels = sess.run([test_image, test_label])
            try:
                test_acc, test_loss=sess.run([test_accuracy2,loss2], feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob2:1.0})
            except tf.errors.OutOfRangeError:
                print("Out of Data ierators")
            acc_sum+=test_acc
        test_acc = acc_sum/test_data_length
        print("Test Acc : %f, Test Loss :%f" %(test_acc,test_loss))
        
    print("Training is done.")
    #Test Model
    save_path = saver.save(sess, "./save/PE92/Basic/Basic_model_PE92.ckpt")
    acc_sum=0.0
    sess.run(test_init_op)
    for epoch in range(total_test_batch):
        batch_images, batch_labels = sess.run([test_image, test_label])
        try:
            test_acc, test_loss=sess.run([test_accuracy2,loss2], feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob2:1.0})
        except tf.errors.OutOfRangeError:
              print("Out of Data ierators")
        acc_sum+=test_acc
    test_acc = acc_sum/test_data_length
    print("Test Acc : %f, Test Loss :%f" %(test_acc,test_loss))
    
    coord.request_stop()
    coord.join(threads)
    train_writer.close()

In [ ]:
train_file_name = "Shuffled_HanDB_Train.tfrecord"
test_file_name = "Shuffled_HanDB_Test.tfrecord"

train_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+train_file_name))
test_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+test_file_name))
total_test_batch = int(test_data_length/batch_size)+1

train_dataset = tf.data.TFRecordDataset("./"+train_file_name)
train_dataset = train_dataset.map(decode)
train_dataset = train_dataset.repeat()
train_dataset= train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.batch(batch_size)


test_dataset = tf.data.TFRecordDataset("./"+test_file_name)
test_dataset = test_dataset.map(decode)
test_dataset = test_dataset.repeat()
test_dataset= test_dataset.shuffle(buffer_size=10000)
test_dataset = test_dataset.batch(batch_size)


train_iterator = tf.data.Iterator.from_structure(train_dataset.output_types, train_dataset.output_shapes)
test_iterator = tf.data.Iterator.from_structure(test_dataset.output_types, test_dataset.output_shapes)

train_init_op = train_iterator.make_initializer(train_dataset)
test_init_op = test_iterator.make_initializer(test_dataset)

train_image, train_label = train_iterator.get_next()
test_image, test_label = test_iterator.get_next()

train_total_batch = int(train_data_length/batch_size)
test_total_batch = int(test_data_length/batch_size)

In [ ]:
init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
saver = tf.train.Saver()

print("Train total Batch :%d" % (train_total_batch))
with tf.Session() as sess:
    
    merged_summary4 = tf.summary.merge([loss_summ3, acc_summ3])
    train_writer = tf.summary.FileWriter("./logs/Basic/HanDB/Train")
    
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    sess.run(train_init_op)
    sess.run(test_init_op)#Test iterator 생성
    #Train Model
    start_time = time.time()
    global_step=0
    for epoch in range(100):
        for i in range(train_total_batch):
        #for i in range(batch_size):
            batch_images, batch_labels = sess.run([train_image, train_label])
            sess.run(optimizer2,feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob2:0.5})
            train_acc, train_loss = sess.run([accuracy2, loss2],feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob2:1.0})
    
            if(i%100==0):
                print('Epoch :%d | step %d, training accuracy: %f, Loss: %f' % (epoch,i, train_acc,train_loss))
                summary = sess.run(merged_summary4, feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob2:1.0})
                train_writer.add_summary(summary,global_step)
                global_step+=1
                
        save_path = saver.save(sess, "./save/HanDB/Basic/Basic_model_HanDB.ckpt")
        end_time = time.time()
        elapsed = end_time - start_time
        
        print("Time : %d : %d: :%f" %(int(elapsed/3600), int((elapsed%3600)/60), (elapsed%60)))
        acc_sum=0.0
        sess.run(test_init_op)
        for epoch in range(total_test_batch):
            batch_images, batch_labels = sess.run([test_image, test_label])
            try:
                test_acc, test_loss=sess.run([test_accuracy2,loss2], feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob2:1.0})
            
            except tf.errors.OutOfRangeError:
                print("Out of Data ierators")
            acc_sum+=test_acc
        test_acc = acc_sum/test_data_length
        print("Test Acc : %f, Test Loss :%f" %(test_acc,test_loss))
   
    print("Training is done.")
    
    acc_sum=0.0
    for epoch in range(total_test_batch):
        batch_images, batch_labels = sess.run([test_image, test_label])
        try:
            test_acc, test_loss=sess.run([test_accuracy2,loss2], feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob2:1.0})
        except tf.errors.OutOfRangeError:
            print("Out of Data ierators")
        acc_sum+=test_acc
    test_acc = acc_sum/test_data_length
    print("Test Acc : %f, Test Loss :%f" %(test_acc,test_loss))
    train_writer.close()
